In [1]:
# Importar pandas
import pandas as pd
import shutil
import os

datasets = {"restaurants":["gijon", "barcelona", "madrid", "paris", "newyorkcity"],
            "pois":["barcelona", "madrid", "paris", "newyorkcity", "london"],
            "amazon":["fashion", "digital_music"]}
models = ["ATT2ITM", "BOW2ITM", "USEM2ITM"]

best_model = pd.read_csv("models/best_models.csv")

for dataset, subsets in datasets.items():
    for subset in subsets:
        subset_results = pd.DataFrame(range(25), columns=["train_dev_items"])
        for model in models:
            md5 = best_model.loc[(best_model.dataset == dataset) & (best_model.subset == subset) & (best_model.model == model)]["model_md5"].values[0]
            results_path = f"models/{model}/{dataset}/{subset}/{md5}/final_eval.csv"
            if os.path.exists(results_path):
                print(results_path)
                os.remove(results_path)
        
                #results = pd.read_csv(results_path)
                #results = results.groupby("cold")["NDCG"].mean().reset_index().rename(columns={"cold":"train_dev_items", "NDCG":model})
                #subset_results = subset_results.merge(results, how="left")
            #else:
                #raise ValueError
        
        #print(subset_results)
        

models/ATT2ITM/restaurants/gijon/544d98b3ba0a0dfd7e3f0019f4d9675e/final_eval.csv
models/BOW2ITM/restaurants/gijon/401e85db0ccb089b9a9c16018ad9e1a4/final_eval.csv
models/USEM2ITM/restaurants/gijon/6176b22e81f5ac3a3fd3d8cef15bf87f/final_eval.csv
models/ATT2ITM/restaurants/barcelona/a751af910be5e885fa3470532027da52/final_eval.csv
models/BOW2ITM/restaurants/barcelona/e7847b82b4b7766d5c55f8c1fa6b420b/final_eval.csv
models/USEM2ITM/restaurants/barcelona/681a8a44718862ca193269a2f26c265b/final_eval.csv
models/ATT2ITM/restaurants/madrid/a4b0c8d943798dec23a1c92e58393f9c/final_eval.csv
models/BOW2ITM/restaurants/madrid/d83ee2225f1cb02abaddf9f363260f4f/final_eval.csv
models/USEM2ITM/restaurants/madrid/8e2850f07e2417da31adda6690ecb102/final_eval.csv
models/ATT2ITM/restaurants/paris/93b1f31e2c35a560da30ecd7415f755e/final_eval.csv
models/BOW2ITM/restaurants/paris/6aea4232acc6d6e60e572d5e8bd9103f/final_eval.csv
models/USEM2ITM/restaurants/paris/f77689739fee40dfa623d0c1e5f8eb58/final_eval.csv
models/AT